# Single neuron dynamics

In [1]:
# note: run this example with Python3 kernel

## libs and path

In [ ]:
try:
    import pyNN.spiNNaker as sim
except ModuleNotFoundError:
    import pyNN.brian2 as sim
    
from pyNN import space 
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pickle
import pandas as pd
import seaborn as sns
import time
import datetime
import json
import warnings
warnings.filterwarnings('ignore')

fileName = 'eg_single-neuron' 
savePath = f'../outputs/'
dt_string = datetime.datetime.today().isoformat() # ISO8601 ! :-)
tag = dt_string 
saveName = f'{savePath}{fileName}-{tag}'
print(saveName)
PARS={}

## sim setup

In [ ]:
PARS['dt']=1 #ms
PARS['min_delay']=1 #ms
PARS['max_delay']=100 #ms
PARS['simtime']=500 #ms

sim.setup(
        timestep=PARS['dt'],
        min_delay=PARS['min_delay'],
        max_delay=PARS['max_delay'])

simtime = PARS['simtime']

## network design

In [ ]:
MODEL=sim.IF_cond_exp
N_CELLS = 1

netPars={'v_rest': -65.0, 
      'v_thresh': -37.0, 
      'v_reset': -80.0,
      'cm': 1.0, 
      'tau_m': 20.0, 
      'tau_refrac': 0.1, 
      'tau_syn_E': 5.0, 
      'tau_syn_I': 5.0, 
      'e_rev_E': 0.0, 
      'e_rev_I': -70.0, 
      'i_offset': 0.0}

cell0 = sim.Population(N_CELLS, 
                      MODEL, 
                      cellparams=netPars,
                      label='cell0')

cell0.record(['v', 'gsyn_exc', 'gsyn_inh', 'spikes'])


## input design

In [ ]:
# not implemented in sPyNNaker
PARS['AC-start']=100 # ms
PARS['AC-stop']=400 # ms
PARS['AC-amplitude']=10 # nA
PARS['AC-frequency']=50 # Hz


pulse = sim.ACSource(start=PARS['AC-start'],
                     stop=PARS['AC-stop'],
                     amplitude=PARS['AC-amplitude'],
                     frequency=PARS['AC-frequency']
                     )

pulse.inject_into(cell0)
pulse.record()



# simulation run

In [ ]:
tic = time.time()
sim.run(PARS['simtime'])
toc = time.time() - tic
print(f'simulation time: {toc} [s]')


# save results

In [ ]:
for recording in ['v', 'gsyn_inh', 'gsyn_exc', 'spikes']:
    cell0.write_data(f'{saveName}-{recording}.pkl')

In [ ]:
cell0.get_data()

In [ ]:
v0=cell0.get_data(variables=['v']).segments[0].analogsignals[0]
ge0=cell0.get_data(variables=['gsyn_exc']).segments[0].analogsignals[0]
gi0=cell0.get_data(variables=['gsyn_inh']).segments[0].analogsignals[0]
spikes0=cell0.get_data(variables=['spikes']).segments[0].spiketrains[0]
i=pulse.get_data()

In [ ]:
PARS['v-mean']=v0.magnitude.mean()
PARS['gsyn_exc-mean']=ge0.magnitude.mean()
PARS['gsyn_inh-mean']=gi0.magnitude.mean()
PARS['fr-mean']=cell0.get_spike_counts()

In [ ]:
JSON=dict(**PARS,**netPars)
with open(f'{saveName}.json', 'w') as json_file:
    json.dump(JSON, json_file)
JSON

In [ ]:
print(f'cell0 spikes: {cell0.get_spike_counts()}')

fig, ax = plt.subplots(1,3, figsize=(13,5))
fig.tight_layout(pad=5)
ax=ax.flatten()
ax[0].plot(i, 'r', alpha=1)
ax[1].plot(v0)
ax[2].plot(ge0, label='ge')
ax[2].plot(gi0, label='gi')

ax[1].axhline(y=cell0[0].v_thresh)
#ax2=ax[1].twinx()
ax2=ax[1].inset_axes([0,1.1,1,0.25])
ax2.set_xlim(0, simtime)
ax2.eventplot(np.array(list(spikes0)).T, color='r', alpha=1, )

ax[0].set_ylabel('[nA]')
ax[0].set_title('ACsource')
ax[1].set_title(f'membrane potential and spikes {cell0.get_spike_counts()}')
ax[2].set_title('conductance')

ax[1].set_ylabel('[mV]')
ax[2].set_ylabel('[nS]')

[ax[i].legend() for i in range(3)]
[ax[i].set_xlim(0, simtime) for i in range(3)]
[ax[i].set_xlabel('[ms]') for i in range(3)]
ax2.set_yticklabels(' ')

fig.savefig(f'{saveName}.jpeg', dpi=100, bbox_inches='tight', facecolor='white')


